## 모델 학습하기
철헌님 코드 활용하였습니다.

In [1]:
# import konlpy and other libraries to analyse Korean text data
from konlpy.tag import Kkma

# create an instance of Kkma and Okt
kkma = Kkma()

In [3]:
import pandas as pd
import numpy as np
import os

from gensim.models import Word2Vec
from konlpy.tag import Okt

import pickle


In [3]:
train_data = pd.read_csv(os.getcwd() + '/sns_syn_sent_augment.csv')

#### 이미 전처리해서 중복 등을 제거한 좋은 데이터로 시작
각 클래스는 정수인코딩 되어있고   
협박 대화 = 0,  
갈취 대화 = 1,  
직장 내 괴롭힘 대화 = 2,  
기타 괴롭힘 대화 = 3,  
일반 = 4   

In [4]:
train_data

,conversation,class
0,선택권이 것인가 너가 네 . 거면 죽여버려야겠군 정말 죽여달라고 혼자 . ? . 선...,0
1,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...,0
2,지금 너 애원하는 것인가 아닙니다 . 죄송합니다 . 죽을 혼자 죽지 우리까지 사건에...,0
3,장난전화는 진정하세요 얘기한다 너에게 달려있다 장난전화 재미있겠지 더 터지면 선생님...,0
4,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...,0
...,...,...
45673,슈룹 시청률 머박! 렬루다가? 치솟았어 어디서 봤어? tvN에서 방영 중이야 시청률...,4
45674,슈룹 시청률 머박! 렬루다가? 어디서 봤어? tvN에서 방영 중이야 시청률이 되는데...,4
45675,도와서 기대해볼게 힐링되면서도 마음 안녕! 봤어 한번 방송이라 얘기하자 가! 의뢰인...,4
45676,이상한 변호사 우영우 너무 재밌어! 응! 나도 매주 기다려! 어제 방송 어땠어? 오...,4


In [5]:
# 레코드의 절반만 가져오기

data_skipped_2 = train_data[train_data.index % 2 == 0] # 22839개 레코드

#### 한국어 형태소 분석 후 word2vec

In [7]:
class0_data = data_skipped_2[data_skipped_2['class'] == 0]
class1_data = data_skipped_2[data_skipped_2['class'] == 1]
class2_data = data_skipped_2[data_skipped_2['class'] == 2]
class3_data = data_skipped_2[data_skipped_2['class'] == 3]
class4_data = data_skipped_2[data_skipped_2['class'] == 4]
print("클래스별 데이터 수")
print("class0_data", len(class0_data))
print("class1_data", len(class1_data))
print("class2_data", len(class2_data))
print("class3_data", len(class3_data))
print("class4_data", len(class4_data))

클래스별 데이터 수
class0_data 1312
class1_data 1466
class2_data 1480
class3_data 1511
class4_data 17070


In [8]:
class0_tokenized_data = [kkma.morphs(sentence) for sentence in class0_data['conversation']]
class1_tokenized_data = [kkma.morphs(sentence) for sentence in class1_data['conversation']]
class2_tokenized_data = [kkma.morphs(sentence) for sentence in class2_data['conversation']]
class3_tokenized_data = [kkma.morphs(sentence) for sentence in class3_data['conversation']]
class4_tokenized_data = [kkma.morphs(sentence) for sentence in class4_data['conversation']]

In [23]:
# 토큰화된 문장들을 리스트에 담는다.
def flatten_token(tokenized_data):
    tokens = []
    for sentence in tokenized_data:
        for word in sentence:
            tokens.append(word)
    print(len(tokens))
    return tokens

class0_tokens = flatten_token(class0_tokenized_data)
class1_tokens = flatten_token(class1_tokenized_data)
class2_tokens = flatten_token(class2_tokenized_data)
class3_tokens = flatten_token(class3_tokenized_data)
class4_tokens = flatten_token(class4_tokenized_data)

1897387


In [40]:
# 가장 자주 보이는 토큰 수를 보여준다. 
from collections import Counter

def get_most_common_tokens(tokens, num_tokens):
    counter = Counter(tokens)
    most_common_tokens = counter.most_common(num_tokens)
    most_common_tokens = [token for token, _ in most_common_tokens]
    return most_common_tokens

class0_most_common_tokens = get_most_common_tokens(class0_tokens, 4000)
class1_most_common_tokens = get_most_common_tokens(class1_tokens, 4000)
class2_most_common_tokens = get_most_common_tokens(class2_tokens, 4000)
class3_most_common_tokens = get_most_common_tokens(class3_tokens, 4000)
class4_most_common_tokens = get_most_common_tokens(class4_tokens, 4000)
# show results in a row
print("class0_most_common_tokens", class0_most_common_tokens)
print("class1_most_common_tokens", class1_most_common_tokens)
print("class2_most_common_tokens", class2_most_common_tokens)
print("class3_most_common_tokens", class3_most_common_tokens)
print("class4_most_common_tokens", class4_most_common_tokens)

In [39]:
# save tokens in dataframe

most_common_tokens = pd.DataFrame(class0_most_common_tokens, columns=['class0_most_common'])
most_common_tokens['class1_most_common'] = class1_most_common_tokens
most_common_tokens['class2_most_common'] = class2_most_common_tokens
most_common_tokens['class3_most_common'] = class3_most_common_tokens
most_common_tokens['class4_most_common'] = class4_most_common_tokens

In [ ]:
most_common_tokens.to_csv(data_dir + '/most-common-tokens.csv', index=False)

In [8]:
# 클래스별 가장 많이 쓰이는 토큰이 저장됨

df = pd.read_csv(os.getcwd()  + '/most-common-tokens.csv')
df.head(20)

,class0_most_common,class1_most_common,class2_most_common,class3_most_common,class4_most_common
0,어,.,.,.,어
1,.,어,하,어,하
2,하,?,?,?,!
3,?,이,이,하,이
4,이,아,어,아,?
5,아,하,아,이,아
6,는,고,는,는,는
7,가,가,가,가,.
8,고,는,고,야,ㄴ
9,!,돈,ㄴ,고,가
